# Using the Vertex AI PaLM API to Generate and Ideate

# Core code
Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

## Setup

In [1]:
from typing import Union
import sys

import os
import io
import json
import base64
import requests
import concurrent.futures
import time

import numpy as np
import pandas as pd

import vertexai.preview.language_models
from google.cloud import aiplatform
from google.cloud import documentai
from google.cloud.documentai_v1 import Document
from google.cloud import storage
from google.cloud import bigquery

from IPython.display import display, Markdown, Latex


In [2]:
PROJECT_ID = 'mg-ce-demos'
REGION = 'us-central1'


In [3]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)


## Ideate with Vertex AI PaLM API

In [11]:
textgen_model = vertexai.preview.language_models.TextGenerationModel.from_pretrained('text-bison@latest')

### Now, we send our prompt to Google GenAI API for some LLM magic

In [24]:
# Provide some context
pre_text = "Pretend you're a marketing specialist, "

# Ask the LLM
text = "Generate a marketing campaign for the launch of a new condominium community in Cincinnati, OH"

prompt = pre_text+text

In [25]:
display(Markdown('## Prompt:'))
print(prompt)
display(Markdown('## Response:'))

# Send prompt to LLM
display(Markdown(str(textgen_model.predict(
   (prompt),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
))))

## Prompt:

Pretend you're a marketing specialist, Generate a marketing campaign for the launch of a new condominium community in Cincinnati, OH


## Response:

**Cincinnati's newest condominium community is now open!**

**The Residences at Liberty Center** is a luxury community located just steps from Liberty Center, a vibrant shopping, dining, and entertainment destination. Our condominiums offer stunning views of the city skyline, spacious layouts, and resort-style amenities.

**Why choose The Residences at Liberty Center?**

* **Luxury living in a convenient location:** Our community is located just steps from Liberty Center, a premier shopping, dining, and entertainment destination. You'll have everything you need right at your fingertips, from world-class shopping to award-winning restaurants to exciting nightlife.
* **Spacious layouts and resort-style amenities:** Our condominiums offer spacious layouts with plenty of room to spread out. And our resort-style amenities include a rooftop pool, fitness center, and concierge service.
* **A welcoming community:** Our community is home to a diverse group of residents who are looking for a vibrant and active lifestyle. We offer a variety of social events and activities, so you're sure to find something to your liking.

**Be one of the first to reserve your new condominium at The Residences at Liberty Center!**

**Visit our website or call us today to learn more.**

**Marketing Campaign**

**Social Media:**

* **Facebook:** Share photos and videos of the community, upcoming events, and resident testimonials. Use the hashtag #ResidencesatLibertyCenter to reach a wider audience.
* **Instagram:** Post stunning photos of the community's amenities and beautiful views. Use the hashtag #ResidencesatLibertyCenter to reach a wider audience.
* **Twitter:** Share news and updates about the community, upcoming events, and special offers. Use the hashtag #ResidencesatLibertyCenter to reach a wider audience.

**Email:**

* Send email newsletters to prospective residents with information about the community, upcoming events, and special offers.
* Create an email drip campaign to nurture leads and encourage them to take the next step in the sales process.

**Website:**

* Create a dedicated landing page for the community with information about the condos, amenities, and location.
* Use a lead generation form on the landing page to collect contact information from prospective residents.

**Paid Advertising:**

* Run paid ads on social media and search engines to target prospective residents in the Cincinnati area.
* Use Google Ads to target people who are searching for condos in the Cincinnati area.
* Use Facebook Ads to target people who are interested in luxury living or who live in the Cincinnati area.

**Public Relations:**

* Send press releases to local media outlets about the community's grand opening.
* Partner with local businesses to host events and giveaways at the community.
* Get involved in the community by volunteering for local organizations or donating to charities.

**By following these marketing strategies, you can reach a wider audience and generate excitement about your new condominium community.**

Voila! We've now used k-means clustering to create groups of spenders and explain their profiles.

Sometimes, though, you want a little bit [extra](https://cloud.google.com/blog/transform/prompt-debunking-five-generative-ai-misconceptions).

In [22]:
def marketing_bot(text):
    pre_text = "Pretend you're a marketing specialist, "
    prompt = pre_text + text
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    
    return prompt, result

A beautiful mountain range with snow-capped peaks and a river running through it. The sun is shining and the sky is blue.  Photorealistic

come up with creative brand persona for each that includes the detail of their favorite movie, a summary of how this relates to their purchasing behavior, and a witty e-mail headline for marketing campaign targeted to their group

## Now we can stick it behind a UI

In [23]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Marketing Bot

    ### Enter a task...

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Task", placeholder="Pretend you're a marketing specialist, ")
        
        
    with gr.Row():
      generate = gr.Button("Generate Response")

    with gr.Row():
      label2 = gr.Textbox(label="Prompt")
    with gr.Row():
      label3 = gr.Textbox(label="Response generated by LLM")

    generate.click(marketing_bot,input_text, [label2, label3])
demo.launch(share=False, debug=False)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
